In [7]:
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

# ---------- paths ----------
DATA = Path("sales_sample.csv")
OUT  = Path("outputs")
OUT.mkdir(exist_ok=True)

# ---------- helpers ----------
EMAIL_RE = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")

COUNTRY_NORMALIZE = {
    "usa": "USA",
    "united states": "USA",
    "pakistan": "Pakistan",
    "pk": "Pakistan",
    "pak": "Pakistan",
    "uae": "UAE",
    "united arab emirates": "UAE",
    "india": "India"
}

def normalize_country(v: str) -> str:
    if pd.isna(v): return v
    key = str(v).strip().lower()
    return COUNTRY_NORMALIZE.get(key, v.strip().title())

def to_number(s):
    """
    Coerce price/qty to numeric:
    - remove $ and commas
    - handle words like 'one' -> 1
    """
    if pd.isna(s): return pd.NA
    s = str(s).strip()
    if s.lower() == "one": return 1
    s = s.replace("$","").replace(",","")
    try:
        return float(s)
    except:
        return pd.NA

def valid_email(s: str) -> bool:
    if pd.isna(s): return False
    return bool(EMAIL_RE.match(str(s).strip()))

def to_datetime_any(s):
    # try multiple formats; errors become NaT
    return pd.to_datetime(s, errors="coerce", dayfirst=True, infer_datetime_format=True) # type: ignore

def export_table_html(df: pd.DataFrame, name: str, caption: str = ""):
    html = df.to_html(index=False)
    if caption:
        html = f"<h3>{caption}</h3>\n" + html
    (OUT / f"{name}.html").write_text(html, encoding="utf-8")

# ---------- load raw ----------
raw = pd.read_csv(DATA, dtype=str)
export_table_html(raw.head(8), "01_before_head", "BEFORE (first 8 rows)")

# ---------- quality report (raw) ----------
qr_raw = pd.DataFrame({
    "Metric": ["Total Rows", "Total Columns", "Missing Values (cells)", "Duplicate Orders (by order_id)"],
    "Value": [
        len(raw),
        raw.shape[1],
        int(raw.isna().sum().sum()),
        int(raw.duplicated(subset=["order_id"]).sum())
    ]
})
export_table_html(qr_raw, "02_quality_report_raw", "QUALITY REPORT (RAW)")

# ---------- cleaning ----------
df = raw.copy()

# strip whitespace
for c in ["customer_name", "email", "country", "channel", "sku"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()

# standardize country
df["country"] = df["country"].apply(normalize_country)

# parse dates
df["order_date"] = df["order_date"].apply(to_datetime_any)

# coerce numerics
df["qty"] = df["qty"].apply(to_number).astype("Float64")
df["unit_price"] = df["unit_price"].apply(to_number).astype("Float64")

# remove exact duplicate order_ids (keep first)
before_dupes = df.shape[0]
df = df.drop_duplicates(subset=["order_id"], keep="first")
after_dupes = df.shape[0]

# compute total
df["total"] = (df["qty"] * df["unit_price"]).round(2)

# email validity flag
df["email_valid"] = df["email"].apply(valid_email)

# ---------- quality report (cleaned) ----------
qr_clean = pd.DataFrame({
    "Metric": [
        "Rows After Dedup",
        "Rows Dropped (dedup)",
        "Rows with NaT order_date",
        "Rows with invalid email",
        "Rows with missing qty or unit_price"
    ],
    "Value": [
        len(df),
        before_dupes - after_dupes,
        int(df["order_date"].isna().sum()),
        int((~df["email_valid"]).sum()),
        int(df["qty"].isna().sum() + df["unit_price"].isna().sum())
    ]
})
export_table_html(qr_clean, "03_quality_report_clean", "QUALITY REPORT (CLEANED)")

# ---------- data dictionary ----------
data_dict = pd.DataFrame([
    ("order_id", "string", "Unique order identifier"),
    ("order_date", "date", "Order date (YYYY-MM-DD)"),
    ("customer_name", "string", "Customer full name"),
    ("email", "string", "Email address"),
    ("country", "string", "Standardized country (e.g., USA, Pakistan, India, UAE)"),
    ("sku", "string", "Product code"),
    ("qty", "number", "Units purchased"),
    ("unit_price", "number", "Price per unit (USD)"),
    ("total", "number", "qty * unit_price"),
    ("channel", "string", "Sales channel (Website/Marketplace)"),
    ("email_valid", "boolean", "Email passes basic regex validation")
], columns=["Column", "Type", "Description"])
export_table_html(data_dict, "04_data_dictionary", "DATA DICTIONARY")

# ---------- AFTER preview ----------
export_table_html(df.head(8), "05_after_head", "AFTER (first 8 rows)")

# ---------- filtered views (for screenshots) ----------
# A) Pakistan orders only
pk = df[df["country"] == "Pakistan"].copy().sort_values("order_date", ascending=False)
export_table_html(pk[["order_id","order_date","customer_name","country","sku","qty","unit_price","total"]],
                  "06_filter_pakistan", "FILTER: Pakistan Orders")

# B) Last 7 days (relative to max date in data)
maxd = df["order_date"].max()
last7 = df[df["order_date"].between(maxd - pd.Timedelta(days=6), maxd)]
export_table_html(last7[["order_id","order_date","country","sku","qty","unit_price","total"]],
                  "07_filter_last7days", "FILTER: Last 7 Days")

# C) Top SKUs by revenue
top_skus = (df.groupby("sku", dropna=False)["total"]
              .sum().sort_values(ascending=False)
              .reset_index().rename(columns={"total":"revenue"}))
export_table_html(top_skus, "08_top_skus", "TOP SKUs by Revenue")

# D) Rows with invalid emails (to show issues found)
bad_email = df[~df["email_valid"]][["order_id","email","country","sku","qty","unit_price","total"]]
export_table_html(bad_email, "09_invalid_emails", "ISSUES: Invalid Emails")

# ---------- save CSVs ----------
raw.to_csv(OUT / "raw_sample_head.csv", index=False)
df.to_csv(OUT / "cleaned_sales.csv", index=False)
pk.to_csv(OUT / "filtered_pakistan.csv", index=False)
last7.to_csv(OUT / "filtered_last7days.csv", index=False)
top_skus.to_csv(OUT / "top_skus.csv", index=False)
bad_email.to_csv(OUT / "invalid_emails.csv", index=False)

print("Done. Open the 'outputs' folder for HTML tables and CSVs.")

website_only = df[df["channel"]=="Website"]
export_table_html(website_only.head(12), "10_filter_website", "FILTER: Website Orders")

(OUT/"sales_report.html").write_text("<h2>UNFILTERED (all rows)</h2>\n"+df.to_html(index=False)+"<hr><h2>FILTERED (Pakistan only)</h2>\n"+pk.to_html(index=False), encoding="utf-8")



C:\Users\DeLL\AppData\Local\Temp\ipykernel_8\509595981.py:51: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", dayfirst=True, infer_datetime_format=True) # type: ignore
C:\Users\DeLL\AppData\Local\Temp\ipykernel_8\509595981.py:51: UserWarning: Parsing dates in %m-%d-%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=True, infer_datetime_format=True) # type: ignore
C:\Users\DeLL\AppData\Local\Temp\ipykernel_8\509595981.py:51: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=

Done. Open the 'outputs' folder for HTML tables and CSVs.


11845896